# side model

In [2]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_2h.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(22445, 2368)
(22445, 7)


6h_dft_dom_cycle_lag39         265
6h_dft_dom_cycle_lag38         262
6h_dft_dom_cycle_lag37         259
6h_dft_dom_cycle_lag36         256
6h_dft_dom_cycle_lag35         253
                              ... 
2h_evenbetter_sinewave_long      0
2h_highpass_bp                   0
2h_comb_spectrum_pwr_24          0
2h_ac_46                         0
2h_ac_12                         0
Length: 2368, dtype: int64

In [3]:
df_features = df_features[265:]
label_side = label_side[265:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(22180, 2368)
(22180, 7)


30m_ac_13             0
2h_voss_lag10         0
2h_voss_lag2          0
2h_voss_lag3          0
2h_voss_lag4          0
                     ..
30m_vwap_dt_lag17     0
30m_vwap_dt_lag18     0
30m_vwap_dt_lag19     0
30m_vwap_dt_lag2      0
6h_williams_r_lag7    0
Length: 2368, dtype: int64

In [4]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(22180, 2368)
(array([0, 1]), array([10356, 11824]))
(array([0, 1]), array([2101, 2335]))


In [5]:

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-24 22:58:35,103] A new study created in memory with name: no-name-1f1a111c-868d-4b8a-ac83-4a411cef6cda
[I 2025-03-24 22:58:47,952] Trial 0 finished with value: 0.8391615290771093 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 153, 'max_depth': 212, 'min_gain_to_split': 0.9777303100910312, 'min_data_in_leaf': 80, 'lambda_l1': 4.723600321852245, 'lambda_l2': 93.24853659742394, 'num_boost_round': 1794}. Best is trial 0 with value: 0.8391615290771093.


precision: 0.751864125932063, recall: 0.7773019271948608, f1: 0.7643714466203412


[I 2025-03-24 22:58:59,845] Trial 1 finished with value: 0.8497572788322477 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 241, 'max_depth': 175, 'min_gain_to_split': 0.02841704083225926, 'min_data_in_leaf': 112, 'lambda_l1': 64.3230164638559, 'lambda_l2': 95.16231025905456, 'num_boost_round': 1086}. Best is trial 1 with value: 0.8497572788322477.


precision: 0.770649895178197, recall: 0.7871520342612419, f1: 0.7788135593220339


[I 2025-03-24 22:59:01,764] Trial 2 finished with value: 0.8461803138507513 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 130, 'max_depth': 72, 'min_gain_to_split': 0.4685150385338584, 'min_data_in_leaf': 308, 'lambda_l1': 89.29785566091819, 'lambda_l2': 28.152769567825246, 'num_boost_round': 102}. Best is trial 1 with value: 0.8497572788322477.


...

precision: 0.7764350453172205, recall: 0.7704496788008566, f1: 0.7734307824591573


[I 2025-03-24 23:15:03,874] Trial 148 finished with value: 0.8517846605114113 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 237, 'max_depth': 379, 'min_gain_to_split': 0.07205783567633187, 'min_data_in_leaf': 489, 'lambda_l1': 14.965485743096588, 'lambda_l2': 63.79427842436245, 'num_boost_round': 203}. Best is trial 91 with value: 0.8544225396899814.


precision: 0.774564015312633, recall: 0.7798715203426124, f1: 0.7772087067861716


[I 2025-03-24 23:15:10,836] Trial 149 finished with value: 0.8491347548378615 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 141, 'max_depth': 403, 'min_gain_to_split': 0.1628326277171684, 'min_data_in_leaf': 475, 'lambda_l1': 5.911979661349624, 'lambda_l2': 73.2271826641078, 'num_boost_round': 302}. Best is trial 91 with value: 0.8544225396899814.


precision: 0.7641628199748216, recall: 0.7798715203426124, f1: 0.7719372615515049


In [7]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 162,
 'max_depth': 42,
 'min_gain_to_split': 0.06825698589791583,
 'min_data_in_leaf': 259,
 'lambda_l1': 12.789929654796069,
 'lambda_l2': 83.31013302587483,
 'num_boost_round': 238}

In [8]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [9]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8544225396899814


In [11]:
model.save_model("custom_indicators/models/model_side.txt")
prod_model.save_model("custom_indicators/models/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_2h.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

6h_dft_dom_cycle_dt_lag27    231
6h_dft_dom_cycle_dt_lag26    228
6h_dft_dom_cycle_lag26       225
6h_dft_dom_cycle_lag25       222
6h_dft_dom_cycle_lag24       219
                            ... 
2h_comb_spectrum_pwr_28        0
2h_comb_spectrum_pwr_27        0
2h_comb_spectrum_pwr_26        0
2h_comb_spectrum_pwr_25        0
side_model_res                 0
Length: 2368, dtype: int64

In [2]:
df_features = df_features[231:]
meta_label = meta_label[231:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(22214, 2368)
(22214, 7)


30m_ac_32                       0
6h_acc_swing_index_ddt_lag12    0
6h_acc_swing_index_ddt_lag2     0
6h_acc_swing_index_ddt_lag33    0
6h_acc_swing_index_dt           0
                               ..
2h_adaptive_bp_lag29            0
2h_adaptive_bp_lag3             0
2h_adaptive_bp_lag30            0
2h_adaptive_bp_lag31            0
side_model_res                  0
Length: 2368, dtype: int64

In [3]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(22214, 2368)
(array([0, 1]), array([ 3302, 14469]))
(array([0, 1]), array([1038, 3405]))


In [4]:

import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.3, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-24 23:40:08,318] A new study created in memory with name: no-name-ceac746d-a92e-4977-b670-6c0718f2c5ac
[I 2025-03-24 23:40:15,852] Trial 0 finished with value: 0.7015883515538371 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 60, 'max_depth': 375, 'min_gain_to_split': 0.4345751572154825, 'min_data_in_leaf': 195, 'lambda_l1': 77.35586501977592, 'lambda_l2': 83.06543824526666, 'num_boost_round': 1063}. Best is trial 0 with value: 0.7015883515538371.


precision: 0.8461276015011941, recall: 0.7283406754772394, f1: 0.7828282828282829, auc: 0.7311669057461118, fbeta: 0.7015883515538371


[I 2025-03-24 23:40:26,754] Trial 1 finished with value: 0.7022349776236119 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 327, 'max_depth': 592, 'min_gain_to_split': 0.5451398565525303, 'min_data_in_leaf': 208, 'lambda_l1': 35.541819912282236, 'lambda_l2': 85.306444350219, 'num_boost_round': 1313}. Best is trial 1 with value: 0.7022349776236119.


precision: 0.842654986522911, recall: 0.7345080763582966, f1: 0.7848736858622313, auc: 0.7254946398105473, fbeta: 0.7022349776236119


[I 2025-03-24 23:40:34,859] Trial 2 finished with value: 0.7212936702288498 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 181, 'max_depth': 469, 'min_gain_to_split': 0.31355149584581987, 'min_data_in_leaf': 178, 'lambda_l1': 24.1599775309331, 'lambda_l2': 61.06446207585141, 'num_boost_round': 328}. Best is trial 2 with value: 0.7212936702288498.


...

precision: 0.8178623718887262, recall: 0.8202643171806168, f1: 0.8190615835777126, auc: 0.7260599424511727, fbeta: 0.7219106965209658


[I 2025-03-25 00:24:17,977] Trial 103 finished with value: 0.7235404019466881 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 181, 'max_depth': 382, 'min_gain_to_split': 0.04079453082618722, 'min_data_in_leaf': 204, 'lambda_l1': 13.549755748633174, 'lambda_l2': 60.957135150484966, 'num_boost_round': 756}. Best is trial 87 with value: 0.7273211378170813.


precision: 0.8152205246468723, recall: 0.8305433186490455, f1: 0.8228105906313645, auc: 0.7285950333720954, fbeta: 0.7235404019466881


[W 2025-03-25 00:24:48,760] Trial 104 failed with parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 140, 'max_depth': 495, 'min_gain_to_split': 0.021827819436454395, 'min_data_in_leaf': 144, 'lambda_l1': 8.631315091050668, 'lambda_l2': 68.67945415846164, 'num_boost_round': 683} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_61183/3410339891.py", line 29, in objective
    model = lgb.train(
            ^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/engine.py", line 322, in train
    booster.update(fobj=fobj)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/lightgbm/

KeyboardInterrupt: 

In [5]:
study.best_params

{'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 193,
 'max_depth': 488,
 'min_gain_to_split': 0.08174521135049798,
 'min_data_in_leaf': 160,
 'lambda_l1': 7.862526652149709,
 'lambda_l2': 40.21459773555627,
 'num_boost_round': 622}

In [6]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("custom_indicators/models/model_meta.txt")
prod_model.save_model("custom_indicators/models/model_meta_prod.txt")